# Part - 1
## Web Scrapping

In [4]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json 
from geopy.geocoders import Nominatim 
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 
!pip install folium
import folium 


     |████████████████████████████████| 92kB 7.0MB/s eta 0:00:011


In [5]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#print(data)

In [6]:
soup = BeautifulSoup(data, 'html.parser')
postalCode = []
borough = []
neighborhood = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    for x in cells:
        postal=x.find_all('b')
        boro=x.find_all('a')
        if len(boro)>0:
            #print(postal[0].text)
            postalCode.append(postal[0].text)
            #print(boro[0].text)
            borough.append(boro[0].text)
            if len(boro)==1:
                neighborhood.append(boro[0].text)
            else:
                st=""
                for i in range(1,len(boro)):
                    st=st+boro[i].text+","
                neighborhood.append(st[:len(st)-2])
            
    
        
df = pd.DataFrame({"PostalCode": postalCode, "Borough": borough, "Neighborhood": neighborhood})

print(len(df))
df.head()

101


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwood
1,M4A,North York,Victoria Villag
2,M5A,Downtown Toronto,"Regent Park,Harbourfron"
3,M6A,North York,"Lawrence Manor,Lawrence Height"
4,M7A,Queen's Park,Ontario Provincial Governmen


In [7]:
df_grouped = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Roug"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Cree"
2,M1E,Scarborough,"Guildwood,Morningside,West Hil"
3,M1G,Scarborough,Wobur
4,M1H,Scarborough,Cedarbra


In [8]:
df_grouped.shape

(101, 3)

# Part - 2
## Marking latitudes and longitudes in the dataframe

In [9]:
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_new = df_grouped.merge(coordinates, on="PostalCode", how="left")
df_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Roug",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Cree",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hil",43.763573,-79.188711
3,M1G,Scarborough,Wobur,43.770992,-79.216917
4,M1H,Scarborough,Cedarbra,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Villag,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park,Ionview,Birchmount Par",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridg",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Villag",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffsid",43.692657,-79.264848


# Part - 3
## Clustering

In [15]:
df_toronto = df_new[df_new['Borough']=='Downtown Toronto']
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
49,M4W,Downtown Toronto,Rosedal,43.679563,-79.377529
50,M4X,Downtown Toronto,"St. James Town,Cabbagetow",43.667967,-79.367675
51,M4Y,Downtown Toronto,Church and Wellesle,43.665860,-79.383160
52,M5A,Downtown Toronto,"Regent Park,Harbourfron",43.654260,-79.360636
53,M5B,Downtown Toronto,"Garden District,Ryerso",43.657162,-79.378937


In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
# create map of Downtown Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto